In [18]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.utils import compute_class_weight

import matplotlib.pyplot as plt

import tarfile

import functools
import pathlib
import shutil
import requests
from tqdm.auto import tqdm
import requests
import zipfile

CALTECH_256_DIR = '256_ObjectCategories'
CALTECH_101_DIR = '101_ObjectCategories'

import tensorflow as tf
from tensorflow import keras
import keras.backend as K
from keras import layers
from keras import activations
from keras.preprocessing.image import ImageDataGenerator
#from keras.utils import to_categorical
from keras.regularizers import l2

from random import seed
from random import random
seed(1)


os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [19]:
! cd 256_ObjectCategories/train && find . -mindepth 1 -maxdepth 1 -type d | grep 101 | wc -l

31


In [20]:
image_size_y = 64
image_size_x = 64
output_size = 257
image_depth = 3

In [21]:
def download(url: str, filename: str):
    """ Util function for downloading file with a loading bar and saves to folder
    
    
    Args:
        url: url to fetch
        filename: file name to save data to
    """    
    r = requests.get(url, stream=True, allow_redirects=True)
    if r.status_code != 200:
        r.raise_for_status()  # Will only raise for 4xx codes, so...
        raise RuntimeError(f"Request to {url} returned status code {r.status_code}")
    file_size = int(r.headers.get("Content-Length", 0))

    path = pathlib.Path(filename).expanduser().resolve()
    path.parent.mkdir(parents=True, exist_ok=True)

    desc = "(Unknown total file size)" if file_size == 0 else ""
    r.raw.read = functools.partial(
        r.raw.read, decode_content=True
    )  # Decompress if needed
    with tqdm.wrapattr(r.raw, "read", total=file_size, desc=desc) as r_raw:
        with path.open("wb") as f:
            shutil.copyfileobj(r_raw, f)

    return path


def fetch_caltech256():
    """Fetches caltech 256 dataset and extracts to 256_ObjectCategories"""
    if not os.path.isfile("data.tar"):
        download(
            "https://data.caltech.edu/tindfiles/serve/813641b9-cb42-4e21-9da5-9d24a20bb4a4",
            "data.tar",
        )
    if not os.path.isdir("256_ObjectCategories"):
        with tarfile.open("data.tar") as tar:
            print('Extracting data tar...')
            tar.extractall()
        if os.path.isdir(CALTECH_256_DIR + '/056.dog/greg'):
            os.rmdir(CALTECH_256_DIR + '/056.dog/greg/vision309')
            os.rmdir(CALTECH_256_DIR + '/056.dog/greg/')

def fetch_caltech101():
    """Fetches caltech 256 dataset and extracts to 256_ObjectCategories"""
    if not os.path.isfile("data.zip"):
        download(
            "https://data.caltech.edu/tindfiles/serve/e41f5188-0b32-41fa-801b-d1e840915e80/",
            "data.zip",
        )
    if not os.path.isdir("caltech-101"):
        with zipfile.ZipFile('data.zip', 'r') as zip_:
            print('Extracting zip...')
            zip_.extractall()
    if not os.path.isdir(CALTECH_101_DIR):
        with tarfile.open("caltech-101/101_ObjectCategories.tar.gz") as tar:
            print('Extracting data tar...')
            tar.extractall()
        with tarfile.open("caltech-101/Annotations.tar") as tar:
            print('Extracting annotations tar...')
            tar.extractall()
            print('Done')

def create_dataset(data_dir: str):
    """Creates a tensroflow dataset that preprocesses images.
    
    Preprocessing done is: 
      * resizing images to 128x128 
      * add padding
    """
    class_names = np.array([""] + sorted(os.listdir(data_dir)))

    def parse_image(filename):
        parts = tf.strings.split(filename, os.sep)
        label = parts[-2]
        oneHot = label == class_names
        encodedLabel = tf.argmax(oneHot)

        image = tf.io.read_file(filename)
        image = tf.io.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, size=(image_size_y, image_size_x)) #64
        #image = tf.image.resize_with_pad(image, target_height=image_size_y, target_width=image_size_x)
        image = image / 256

        return image, encodedLabel

    list_ds = tf.data.Dataset.list_files(data_dir + "/*/*.jpg", shuffle=True)

    return list_ds.map(parse_image)


def move_files(files: list, destination_folder: str):
    """Helper file to move files arouond"""
    if not os.path.isdir(destination_folder):
        os.makedirs(destination_folder)

    for file_ in files:
        destination = os.path.join(destination_folder, os.path.basename(file_))
        os.rename(file_, destination)


def split_data(data_dir):
    print('Splitting data...')
    for folder, folders, files in list(os.walk(data_dir))[1:]:
        files.sort()

        files = [os.path.join(folder, file_) for file_ in files]

        n_files = len(files)
        n_train = n_files // 2
        n_val = n_files // 4
        n_test = n_files - n_train - n_val

        i_train = 0
        i_val = i_train + n_train
        i_test = i_val + n_val

        move_files(
            files[:i_val], os.path.join(data_dir, "train", os.path.basename(folder))
        )
        move_files(
            files[i_val:i_test], os.path.join(data_dir, "val", os.path.basename(folder))
        )
        move_files(
            files[i_test:], os.path.join(data_dir, "test", os.path.basename(folder))
        )

        if not os.listdir(folder):
            os.rmdir(folder)


def create_datasets(is_101=False):
    if is_101:
        fetch_caltech101()
        data_dir = CALTECH_101_DIR
    else:
        fetch_caltech256()
        data_dir = CALTECH_256_DIR

    print('Loading data...')

    if not os.path.isdir(data_dir + "/train"):
        split_data(data_dir)

    train_x, train_y = zip(*create_dataset(data_dir + "/train").as_numpy_iterator())
    val_x, val_y = zip(*create_dataset(data_dir + "/val").as_numpy_iterator())
    test_x, test_y = zip(*create_dataset(data_dir + "/test").as_numpy_iterator())
    print('Done')

    return train_x, train_y, val_x, val_y, test_x, test_y

def create_datasets_tf(is_101=False):
    if is_101:
        fetch_caltech101()
        data_dir = CALTECH_101_DIR
    else:
        fetch_caltech256()
        data_dir = CALTECH_256_DIR

    if not os.path.isdir(data_dir + "/train"):
        split_data()

    train = create_dataset(data_dir + "/train")
    val = create_dataset(data_dir + "/val")
    test = create_dataset(data_dir + "/test")

    return train, val, test

def get_label_name(label: int, is_101=False):
    """Gets the name associated with a label""" 
    if is_101:
        data_dir = CALTECH_101_DIR
    else:
        data_dir = CALTECH_256_DIR

    dirs = os.listdir(data_dir + "/train")
    return sorted(dirs)[label - 1].split(".")[1]

def get_label_for_name(name: str, is_101=False):
    if is_101:
        data_dir = CALTECH_101_DIR
    else:
        data_dir = CALTECH_256_DIR

    search_list = [f.split('.')[1] for f in sorted(os.listdir(data_dir  + '/train'))]
    return search_list.index(name) + 1

In [22]:
# Class for the data generator with autoencoder generator output
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    
    # Input arguments are as follows:
    #  x:            our X data array that we'll augment
    #  y_one:        y labels for the first output
    #  y_two:        y labels for the second ouput
    #  data_aug:     our data augmentor
    #  batch_size:   the batch size to return from the generator
    #  dim:          size of images
    #  n_channels:   number of image channels
    #  shuffle:      flag to indicate if we should shuffle the data at the end of the epoch
    def __init__(self, x, y_one, y_two, data_aug,                
                 batch_size=32, dim=(64,64), n_channels=3,
                 shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.x = x
        self.y_one = y_one
        self.y_two = y_two
        self.data_aug = data_aug
        self.list_IDs = np.arange(0, self.x.shape[0])
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            

    # Function to generate data. This will take in a list of sample indicies, and then for each
    # these apply augmentation, and return the augmented data and labels
    #
    # If you wish to check the mechanics out in more detail, you can uncomment the two print lines
    # which will show you the IDs that are being manipulated and help show what's going on - however
    # this will also generate a lot of output text during model training.
    #
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization - creating space for X and y data
#        print(list_IDs_temp)
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        # creating a list of empty arrays to hold our data labels. 
        # would initialise this for the number of output labels we have
        y = [np.empty((self.batch_size), dtype=int),
             np.empty((self.batch_size), dtype=int),
             np.empty((self.batch_size, *self.dim, self.n_channels))]

        # Generate data, loop through the list of IDs we have and generate data for each in turn
        for i, ID in enumerate(list_IDs_temp):
            #print(ID)
            randomNumber = int(999999999 * random())
            # apply random transformation based of our datagen instance
            X[i,] = self.data_aug.random_transform(self.x[ID,], randomNumber)
            # copy our y labels across
            y[0][i,] = self.y_one[ID]
            y[1][i,] = self.y_two[ID]
            y[2][i,] = X[i,]
        return X, y

In [23]:
train_x, train_y, val_x, val_y, test_x, test_y = create_datasets(is_101=False)

Loading data...
Done


In [24]:
#for i in range(257):
#    print(str(i) + " is " + get_label_name(i))

ignore = [0, 82, 159]
plant = [15, 25, 68, 92, 103, 147, 154, 204, 221, 242]
friend = [40, 159, 253]

new_classes = ['ignore', 'plant', 'friend', 'garbage']

def select_garbage(data): 
    new_data = []
    for x in data:
        converted_to = 3
        if x in ignore:
            converted_to = 0
        elif x in plant:
            converted_to = 1
        elif x in friend:
            converted_to = 2
        
        new_data.append(converted_to)
    #print(str(x) + " is now " + str())
    return new_data

In [25]:
train_x_new = np.asarray(train_x)
train_y_new = np.asarray(train_y)
train_y_new = train_y_new-1
train_garbage = np.asarray(select_garbage(train_y_new))
val_x_new = np.asarray(val_x)
val_y_new = np.asarray(val_y)
val_y_new  = val_y_new -1
val_garbage = np.asarray(select_garbage(val_y_new))
test_x_new = np.asarray(test_x)
test_y_new = np.asarray(test_y)
test_y_new = test_y_new-1
test_garbage = np.asarray(select_garbage(test_y_new))


In [26]:
def focal_loss(target, output, gamma=2):
    output /= K.sum(output, axis=-1, keepdims=True)
    eps = K.epsilon()
    output = K.clip(output, eps, 1. - eps)
    return -K.sum(K.pow(1. - output, gamma) * target * K.log(output), axis=-1)   

In [27]:
# Ad noted above: these params are not suitable, just used to demo it in practice
data_aug = ImageDataGenerator(
                            # rotate between -10, +10 degrees
                            rotation_range=4,
                            # horiziontal shift by +/- 10% of the image width
                            width_shift_range=0.05,
                            # vertical shift by +/- 10% of the image width
                            height_shift_range=0.05,
                            # +/- 10% range for randomly applying a shearing transform
                            #shear_range=0.05,
                            # +/- 10% range for zooming
                            zoom_range=0.2,
                            # allow horizontal flips of data
                            #horizontal_flip=True,
                            # what value to place in new pixels, given the nature of our data
                            # extend images
                            fill_mode='nearest')

In [28]:
datagen = DataGenerator(train_x_new, train_y_new, train_garbage, data_aug, batch_size=64, dim=(image_size_y, image_size_x), n_channels=image_depth)

In [29]:
def getVGGBlock(x, filter):
     # run pairs of conv layers, all 3s3 kernels
    x = layers.Conv2D(filters=filter, kernel_size=(3,3), padding='same', activation='relu')(x)
    x = layers.Conv2D(filters=filter, kernel_size=(3,3), padding='same', activation=None)(x)
    # batch normalisation, before the non-linearity
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    # spatial dropout, this will drop whole kernels, i.e. 20% of our 3x3 filters will be dropped out rather
    # than dropping out 20% of the invidual pixels
    x = layers.SpatialDropout2D(0.2)(x)
    # max pooling, 2x2, which will downsample the image
    x = layers.MaxPool2D(pool_size=(2, 2))(x)
    return x

def getFCCBlock(x, filter):
    x = layers.Dense(filter, activation=None)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.2)(x)
    return x

#DCNN model needs to match the MNIST
def get_DCNN_model(upscaling_bool=False, global_pool=False, filters=[16, 32, 64], fcd=[1024, 512], pretrained=False):
    # our model, input in an image shape
    input_t = keras.Input(shape=(image_size_y, image_size_x, image_depth))

    if (upscaling_bool):
        x = keras.layers.Lambda(lambda image: tf.image.resize(image, (224, 224)))(input_t)
    else:
        x = input_t

    if (pretrained==False):
        for filter_stage in filters:
            x = getVGGBlock(x, filter_stage)
    else:
        x_model = keras.applications.VGG16(weights='imagenet', include_top=False, input_tensor=x)
        for layer in x_model.layers[:-5]:
            layer.trainable = False
        x = x_model.output
        
    pre_x = x
    # final conv2d, batch norm and spatial dropout
    # flatten layer
    if global_pool:
        x = layers.GlobalAveragePooling2D()(x)

    x = layers.Flatten()(x)
    # we'll use a couple of dense layers here, mainly so that we can show what another dropout layer looks like 
    # in the middle
    x1 = x
    for fcc in fcd:
        x1 = getFCCBlock(x1, fcc)
    output_class = layers.Dense(257, activation='softmax', name="class")(x1)

    x2 = x
    for fcc in fcd:
        x2 = getFCCBlock(x2, fcc)
    output_garbage = layers.Dense(4, activation='softmax', name="garbage")(x2)

    x3 = layers.UpSampling2D((2, 2))(pre_x)
    x3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x3)
    x3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x3)
    
    x3 = layers.UpSampling2D((2, 2))(x3)
    x3 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x3)
    x3 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x3)
    
    x3 = layers.UpSampling2D((2, 2))(x3)
    x3 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x3)
    x3 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x3)
    x3 = layers.Conv2D(3, (1, 1), activation='softmax', padding='same')(x3)
    x3 = keras.layers.Lambda(lambda image: tf.image.resize(image, (image_size_y, image_size_x)))(x3)
    output_decoded = layers.Conv2D(3, (1, 1), activation='softmax', padding='same', name="decoded")(x3)
    # the output
    model = keras.Model(inputs=input_t, outputs=[output_class, output_garbage, output_decoded], name='Ass1cQ2')
    model.summary()
    return model

In [30]:
def update_trainable_layers(model, layers_to_train_local, norm_on=False):
    #pretrained_model.summary()
    for layer in model.layers[-21].layers:
        layer.trainable = False
    if (layers_to_train_local != 0):
        for layer in model.layers[-21].layers[-layers_to_train_local:]:
            layer.trainable = True
            if (norm_on == False):
                if (isinstance(layer, layers.BatchNormalization)):
                    layer.trainable = False
    for i, layer in enumerate(model.layers[-21].layers):
        if layer.trainable:
            print(i, layer.name, "-", layer.trainable)
    return model

In [31]:
def evaluate_model(model_history, trained_model):
    fig = plt.figure(figsize=[10, 10])
    ax = fig.add_subplot(1, 3, 1)
    predict_class, predict_garbage, predict_picture = trained_model.predict(test_x_new);
    garbage_indexes = tf.argmax(predict_garbage, axis=1)
    class_indexes = tf.argmax(predict_class, axis=1)
    cm = confusion_matrix(test_garbage, garbage_indexes)
    c = ConfusionMatrixDisplay(cm, display_labels=range(4))
    c.plot(ax = ax)    
    ax.set_title('Testing Loss');
    ax = fig.add_subplot(1, 3, 2)
    ax.plot(model_history.history['loss'], label="Training Loss")
    ax.plot(model_history.history['val_loss'], label="Validation Loss")
    ax.plot(model_history.history['val_garbage_loss'], label="Validation Garbage Loss")
    ax.plot(model_history.history['val_class_loss'], label="Validation Class Loss")
    ax.legend()
    ax.set_title('Testing Accuracy');
    ax = fig.add_subplot(1, 3, 3)
    #ax.plot(model_history.history['accuracy'], label="Training Accuracy")
    #ax.plot(model_history.history['val_accuracy'], label="Validation Accuracy")
    ax.plot(model_history.history['val_garbage_accuracy'], label="Validation Garbage Accuracy")
    ax.plot(model_history.history['val_class_accuracy'], label="Validation Class Accuracy")
    ax.legend()
    print(classification_report(test_garbage, garbage_indexes))
    print(classification_report(test_y_new, class_indexes))

In [32]:

def train_model(iteration, model, local_epoch=150, batch_size_chosen=64):  
     local_steps = np.ceil(10000/batch_size_chosen)
     check_point = keras.callbacks.ModelCheckpoint(filepath="VGG-WallE" + str(iteration) + ".h5",
                                              monitor="val_class_loss",
                                              mode="min",
                                              save_best_only=True,
                                              )
     model.compile(loss=[keras.losses.SparseCategoricalCrossentropy(from_logits=False),keras.losses.SparseCategoricalCrossentropy(from_logits=False),focal_loss],
                  optimizer=keras.optimizers.RMSprop(), metrics=['accuracy'])
     history = model.fit(x=train_x_new, y=[train_y_new, train_garbage, train_x_new], batch_size=batch_size_chosen, epochs=local_epoch, steps_per_epoch=local_steps, verbose=2,
                        validation_data=(val_x_new, [val_y_new, val_garbage, val_x_new]),
                        callbacks=[check_point, keras.callbacks.EarlyStopping(monitor='val_class_accuracy', patience=20)])
    
     return history, model

In [ ]:
model = get_DCNN_model(upscaling_bool=False, global_pool=True, filters=[], fcd=[], pretrained=True)
model_history, model_trained = train_model(77, model, batch_size_chosen=62)
evaluate_model(model_history, model_trained)

In [33]:
#upscaling_options = [False, True]
#global_pooling = [True, False]
#pretrained_opt = [False, True]
#trainable_layers = [[16, 32, 64],[32, 64, 64, 128],[256, 256, 512],[8, 16, 32, 64],[8, 16, 32],[64, 128, 256],[32, 64, 128, 256, 512],[1024, 1024, 512]]
#trainable_fcc = [[], [512], [1024], [512, 256], [1024, 512]]


In [34]:
#iteration_count = 0
#for upscaling in upscaling_options:
#    for pooling in global_pooling:
#        for fcc_arr in trainable_fcc:
#            print(str(upscaling) + " " + str(pooling) + " " + str(fcc_arr))
#            model = get_DCNN_model(upscaling_bool=upscaling, global_pool=pooling, filters=[], fcd=fcc_arr, pretrained=True)
#            model_history, model_trained = train_model(iteration_count, model, batch_size_chosen=62)
#            evaluate_model(model_history, model_trained)
#            iteration_count += 1

False True []
Model: "Ass1cQ2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 64, 64, 64)   1792        ['input_2[0][0]']                
                                                                                                  
 block1_conv2 (Conv2D)          (None, 64, 64, 64)   36928       ['block1_conv1[0][0]']           
                                                                                                  
 block1_pool (MaxPooling2D)     (None, 32, 32, 64)   0           ['block1_conv2[0][0]']           
                                                                              

KeyboardInterrupt: 

In [ ]:

for upscaling in upscaling_options:
    for pooling in global_pooling:
        for filter_arr in trainable_layers:
            for fcc_arr in trainable_fcc:
                print(str(upscaling) + " " + str(pooling) + " " + str(filter_arr) + " " + str(fcc_arr))
                model = get_DCNN_model(upscaling_bool=upscaling, global_pool=pooling, filters=filter_arr, fcd=fcc_arr, pretrained=False)
                model_history, model_trained = train_model(iteration_count, model, batch_size_chosen=62)
                evaluate_model(model_history, model_trained)
                iteration_count += 1

In [ ]:
#winner = 0
#winner_value = 0
#for layer_count in range(0, 234):
#    model = update_trainable_layers(model, layer_count, norm_on=True)
#    print("Layer Count: " + str(layer_count))
#    model_history, model = train_model(iteration_count, model, batch_size_chosen=32)
#    print("Accuracy: " + str(model_history.history['val_class_accuracy'][-1]))
#    
#    if (model_history.history['val_class_accuracy'][-1] > winner_value):
#        winner = iteration_count
#        winner_value = model_history.history['val_class_accuracy'][-1]
#        print("###########################NEW WINNER###############################")
#    iteration_count += 1
#evaluate_model(model_history, model)

In [ ]:
#absl-py>=0.15.0
#argon2-cffi>=21.1.0
#astunparse>=1.6.3
#attrs>=21.2.0
#backcall>=0.2.0
#bleach>=4.1.0
#cachetools>=4.2.4
#certifi>=2021.10.8
#cffi>=1.15.0
#charset-normalizer>=2.0.7
#clang>=5.0
#cycler>=0.11.0
#debugpy>=1.5.1
#decorator>=5.1.0
#defusedxml>=0.7.1
#entrypoints>=0.3
#flatbuffers>=1.12
#gast>=0.4.0
#gensim>=4.1.2
#google-auth>=2.3.2
#google-auth-oauthlib>=0.4.6
#google-pasta>=0.2.0
#grpcio>=1.41.1
#h5py>=3.1.0
#idna>=3.3
#imageio>=2.10.1
#ipykernel>=6.4.2
#ipython>=7.28.0
#ipython-genutils>=0.2.0
#ipywidgets>=7.6.5
#jedi>=0.18.0
#Jinja2>=3.0.2
#joblib>=1.1.0
#jsonschema>=4.1.2
#jupyter>=1.0.0
#jupyter-client>=7.0.6
#jupyter-console>=6.4.0
#jupyter-core>=4.9.1
#jupyterlab-pygments>=0.1.2
#jupyterlab-widgets>=1.0.2
#keras>=2.6.0
#Keras-Preprocessing>=1.1.2
#kiwisolver>=1.3.2
#Markdown>=3.3.4
#MarkupSafe>=2.0.1
#matplotlib>=3.4.3
#matplotlib-inline>=0.1.3
#mistune>=0.8.4
#nbclient>=0.5.4
#nbconvert>=6.2.0
#nbformat>=5.1.3
#nest-asyncio>=1.5.1
#networkx>=2.6.3
#notebook>=6.4.5
#numpy>=1.19.5
#oauthlib>=3.1.1
#opencv-python>=4.5.4.58
#opt-einsum>=3.3.0
#packaging>=21.0
#pandas>=1.3.4
#pandocfilters>=1.5.0
#parso>=0.8.2
#patsy>=0.5.2
#pexpect>=4.8.0
#pickleshare>=0.7.5
#Pillow>=8.4.0
#prometheus-client>=0.11.0
#prompt-toolkit>=3.0.21
#protobuf>=3.19.1
#ptyprocess>=0.7.0
#pyasn1>=0.4.8
#pyasn1-modules>=0.2.8
#pycparser>=2.20
#pydot>=1.4.2
#Pygments>=2.10.0
#pyparsing>=3.0.3
#pyrsistent>=0.18.0
#python-dateutil>=2.8.2
#pytz>=2021.3
#PyWavelets>=1.1.1
#pyzmq>=22.3.0
#qtconsole>=5.1.1
#QtPy>=1.11.2
#requests>=2.26.0
#requests-oauthlib>=1.3.0
#rsa>=4.7.2
#scikit-image>=0.18.3
#scikit-learn>=1.0.1
#scipy>=1.7.1
#seaborn>=0.11.2
#Send2Trash>=1.8.0
#six>=1.15.0
#smart-open>=5.2.1
#statsmodels>=0.13.1
#tensorboard>=2.7.0
#tensorboard-data-server>=0.6.1
#tensorboard-plugin-wit>=1.8.0
#tensorflow>=2.6.0
#tensorflow-estimator>=2.6.0
#termcolor>=1.1.0
#terminado>=0.12.1
#testpath>=0.5.0
#threadpoolctl>=3.0.0
#tifffile>=2021.10.12
#tornado>=6.1
#tqdm>=4.64.0
#traitlets>=5.1.1
#typing-extensions>=3.7.4.3
#urllib3>=1.26.7
#wcwidth>=0.2.5
#webencodings>=0.5.1
#Werkzeug>=2.0.2
#widgetsnbextension>=3.5.2
#wrapt>=1.12.1
